In [1]:
import pandas as pd
import pymongo
from pymongo import MongoClient
import json
import re
import os
import sys
import geopandas as gpd
from dotenv import load_dotenv
import extract_to_mongodb as etm
import db_utils as dbu



In [2]:
# Step 1: Load CSV files into DataFrames
#/workspaces/WildFireSimulation/data/result/geocoding_chunk_10.csv
file_paths = [f'/workspaces/WildFireSimulation/data/result/geocoding_chunk_{i}.csv' for i in range(0, 15)]
dataframes = [pd.read_csv(file_path) for file_path in file_paths]


/tmp/ipykernel_18287/3589123073.py:4: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = [pd.read_csv(file_path) for file_path in file_paths]


In [3]:
# Add the 'scripts' directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

In [4]:
#code tells that we are connected to scripts
print(dir(dbu))

['ConnectionFailure', 'MongoClient', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'db_name', 'gpd', 'insert_data_to_mongodb', 'insert_dataframe_to_mongodb', 'insert_df_only_to_mongodb', 'load_data_from_mongodb', 'load_dotenv', 'mongo_uri', 'os', 'pd', 'read_data_from_mongodb']


In [5]:
# Step 2: Concatenate all DataFrames into one
merged_df = pd.concat(dataframes, ignore_index=True)

In [6]:
merged_df.head()

,_id,lat,lon,rep_date,source,sensor,satellite,agency,temp,rh,...,cfl,tfc0,ecozone,sfc0,geometry,country,municipality,postalCode,freeformAddress,addressComponents
0,6669b15b44a41007063ba9da,62.1163,-161.8260,2020-06-02,NASA4,IBAND,S-NPP,UAK,15.917,69,...,0.000000,0.35,2,0.35,"{'type': 'Point', 'coordinates': [-161.826, 62...",NaN,NaN,NaN,NaN,"{""address"": {""routeNumbers"": [], ""countryCode""..."
1,6669b15e44a41007063ba9db,61.6291,-164.4490,2020-06-11,NASA5,IBAND,NOAA-20,UAK,26.064,52,...,0.000000,0.10,2,0.10,"{'type': 'Point', 'coordinates': [-164.449, 61...",NaN,NaN,NaN,NaN,"{""address"": {""routeNumbers"": [], ""countryCode""..."
2,6669b15e44a41007063ba9dc,47.4740,-69.9380,2020-06-20,USFS,IBAND,JPSS1,QC,24.666,54,...,0.000000,1.36,NaN,1.36,"{'type': 'Point', 'coordinates': [-69.938, 47....",NaN,NaN,NaN,NaN,"{""address"": {""buildingNumber"": ""23"", ""streetNu..."
3,6669b15e44a41007063ba9dd,32.3920,-110.8040,2020-06-22,NASA6,IBAND,S-NPP,UAZ,32.754,11,...,0.000530,0.35,12,0.35,"{'type': 'Point', 'coordinates': [-110.804, 32...",NaN,NaN,NaN,NaN,"{""address"": {""buildingNumber"": ""10000"", ""stree..."
4,6669b15e44a41007063ba9de,33.4720,-91.3332,2020-06-13,NASA7,IBAND,NOAA-20,UAR,31.359,37,...,0.000499,0.35,8,0.35,"{'type': 'Point', 'coordinates': [-91.3332, 33...",NaN,NaN,NaN,NaN,"{""address"": {""buildingNumber"": ""300"", ""streetN..."


In [7]:
merged_df.count()

_id                  449139
lat                  449139
lon                  449139
rep_date             449139
source               449139
sensor               449139
satellite            448902
agency               449139
temp                 449139
rh                   449139
ws                   449139
wd                   449139
pcp                  449139
ffmc                 449139
dmc                  449139
dc                   449139
isi                  449139
bui                  449139
fwi                  449139
fuel                 449139
ros                  449136
sfc                  449136
tfc                  449136
bfc                  238067
hfi                  449136
cfb                  449136
pcuring              424857
greenup              424857
elev                 449139
sfl                  449139
cfl                  449139
tfc0                 449136
ecozone              448129
sfc0                 449136
geometry             449139
country             

In [8]:
# Function to extract municipality
def extract_municipality(address_component):
    try:
        # Parse the JSON object
        address = json.loads(address_component)
        return address['address'].get('municipality', None)
    except (json.JSONDecodeError, KeyError):
        return None


In [9]:
# Function to extract postal code
def extract_postal_code(address_component):
    try:
        # Parse the JSON object
        address = json.loads(address_component)
        return address['address'].get('postalCode', None)
    except (json.JSONDecodeError, KeyError):
        return None

In [10]:
# Function to extract free form address
def extract_free_form_address(address_component):
    try:
        # Parse the JSON object
        address = json.loads(address_component)
        return address['address'].get('freeformAddress', None)
    except (json.JSONDecodeError, KeyError):
        return None

In [11]:
# Function to extract free form address
def extract_country(address_component):
    try:
        # Parse the JSON object
        address = json.loads(address_component)
        return address['address'].get('countryCode', None)
    except (json.JSONDecodeError, KeyError):
        return None

In [12]:
# Apply extraction functions to populate columns
merged_df['country'] = merged_df['addressComponents'].apply(extract_country)
merged_df['municipality'] = merged_df['addressComponents'].apply(extract_municipality)
merged_df['postalCode'] = merged_df['addressComponents'].apply(extract_postal_code)
merged_df['freeformAddress'] = merged_df['addressComponents'].apply(extract_free_form_address)


In [13]:
merged_df.count()

_id                  449139
lat                  449139
lon                  449139
rep_date             449139
source               449139
sensor               449139
satellite            448902
agency               449139
temp                 449139
rh                   449139
ws                   449139
wd                   449139
pcp                  449139
ffmc                 449139
dmc                  449139
dc                   449139
isi                  449139
bui                  449139
fwi                  449139
fuel                 449139
ros                  449136
sfc                  449136
tfc                  449136
bfc                  238067
hfi                  449136
cfb                  449136
pcuring              424857
greenup              424857
elev                 449139
sfl                  449139
cfl                  449139
tfc0                 449136
ecozone              448129
sfc0                 449136
geometry             449139
country             

In [14]:
merged_df

,_id,lat,lon,rep_date,source,sensor,satellite,agency,temp,rh,...,cfl,tfc0,ecozone,sfc0,geometry,country,municipality,postalCode,freeformAddress,addressComponents
0,6669b15b44a41007063ba9da,62.116300,-161.826000,2020-06-02,NASA4,IBAND,S-NPP,UAK,15.917000,69,...,0.000000,0.35,2,0.35,"{'type': 'Point', 'coordinates': [-161.826, 62...",US,Yukon Delta Wilderness,None,"Yukon Delta Wilderness, AK","{""address"": {""routeNumbers"": [], ""countryCode""..."
1,6669b15e44a41007063ba9db,61.629100,-164.449000,2020-06-11,NASA5,IBAND,NOAA-20,UAK,26.064000,52,...,0.000000,0.10,2,0.10,"{'type': 'Point', 'coordinates': [-164.449, 61...",US,Chevak,None,"Chevak, AK","{""address"": {""routeNumbers"": [], ""countryCode""..."
2,6669b15e44a41007063ba9dc,47.474000,-69.938000,2020-06-20,USFS,IBAND,JPSS1,QC,24.666000,54,...,0.000000,1.36,NaN,1.36,"{'type': 'Point', 'coordinates': [-69.938, 47....",CA,Saint-Denis-De La Bouteillerie,G0L,"23 Rang du Bras, Saint-Denis-De La Bouteilleri...","{""address"": {""buildingNumber"": ""23"", ""streetNu..."
3,6669b15e44a41007063ba9dd,32.392000,-110.804000,2020-06-22,NASA6,IBAND,S-NPP,UAZ,32.754000,11,...,0.000530,0.35,12,0.35,"{'type': 'Point', 'coordinates': [-110.804, 32...",US,Mount Lemmon,85619,"10000 East Ski Run Road, Mount Lemmon, AZ 85619","{""address"": {""buildingNumber"": ""10000"", ""stree..."
4,6669b15e44a41007063ba9de,33.472000,-91.333200,2020-06-13,NASA7,IBAND,NOAA-20,UAR,31.359000,37,...,0.000499,0.35,8,0.35,"{'type': 'Point', 'coordinates': [-91.3332, 33...",US,Carlton,71653,"300 Cedar Road, Lake Village, AR 71653","{""address"": {""buildingNumber"": ""300"", ""streetN..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449134,6670e69da6dc531d5ed14328,56.340000,-119.229897,2023-05-13,NASA,MODIS,Aqua,AB,25.278999,25,...,0.032534,2.44,9,2.19,"{'type': 'Point', 'coordinates': [-119.229897,...",CA,Worsley,T0H,"Range Road 84, Worsley AB T0H 3W0","{""address"": {""routeNumbers"": [], ""street"": ""Ra..."
449135,6670e69ea6dc531d5ed14329,31.086281,-98.919960,2023-03-26,NASA7,VIIRS-I,NOAA-20,UTX,21.538000,18,...,0.008307,0.35,9,0.35,"{'type': 'Point', 'coordinates': [-98.91996, 3...",US,San Saba,76877,"Deep Creek Road, San Saba, TX 76877","{""address"": {""routeNumbers"": [], ""street"": ""De..."
449136,6670e69ea6dc531d5ed1432a,56.339291,-119.209106,2023-05-13,NASA,MODIS,Aqua,AB,25.422001,25,...,0.217550,2.52,9,2.20,"{'type': 'Point', 'coordinates': [-119.209106,...",CA,Worsley,T0H,"Range Road 84, Worsley AB T0H 3W0","{""address"": {""routeNumbers"": [], ""street"": ""Ra..."
449137,6670e69ea6dc531d5ed1432b,56.049808,-116.394592,2023-05-13,NASA2,VIIRS-I,S-NPP,AB,25.267000,24,...,0.162315,2.38,9,2.00,"{'type': 'Point', 'coordinates': [-116.394592,...",CA,Cadotte Lake,T0H,Cadotte Lake AB T0H 0N0,"{""address"": {""routeNumbers"": [], ""countryCode""..."


In [15]:
print("Shape of all database",merged_df.shape)

Shape of all database (449139, 40)


In [16]:
merged_df.isna().sum()

_id                       0
lat                       0
lon                       0
rep_date                  0
source                    0
sensor                    0
satellite               237
agency                    0
temp                      0
rh                        0
ws                        0
wd                        0
pcp                       0
ffmc                      0
dmc                       0
dc                        0
isi                       0
bui                       0
fwi                       0
fuel                      0
ros                       3
sfc                       3
tfc                       3
bfc                  211072
hfi                       3
cfb                       3
pcuring               24282
greenup               24282
elev                      0
sfl                       0
cfl                       0
tfc0                      3
ecozone                1010
sfc0                      3
geometry                  0
country             

In [17]:
merged_df

,_id,lat,lon,rep_date,source,sensor,satellite,agency,temp,rh,...,cfl,tfc0,ecozone,sfc0,geometry,country,municipality,postalCode,freeformAddress,addressComponents
0,6669b15b44a41007063ba9da,62.116300,-161.826000,2020-06-02,NASA4,IBAND,S-NPP,UAK,15.917000,69,...,0.000000,0.35,2,0.35,"{'type': 'Point', 'coordinates': [-161.826, 62...",US,Yukon Delta Wilderness,None,"Yukon Delta Wilderness, AK","{""address"": {""routeNumbers"": [], ""countryCode""..."
1,6669b15e44a41007063ba9db,61.629100,-164.449000,2020-06-11,NASA5,IBAND,NOAA-20,UAK,26.064000,52,...,0.000000,0.10,2,0.10,"{'type': 'Point', 'coordinates': [-164.449, 61...",US,Chevak,None,"Chevak, AK","{""address"": {""routeNumbers"": [], ""countryCode""..."
2,6669b15e44a41007063ba9dc,47.474000,-69.938000,2020-06-20,USFS,IBAND,JPSS1,QC,24.666000,54,...,0.000000,1.36,NaN,1.36,"{'type': 'Point', 'coordinates': [-69.938, 47....",CA,Saint-Denis-De La Bouteillerie,G0L,"23 Rang du Bras, Saint-Denis-De La Bouteilleri...","{""address"": {""buildingNumber"": ""23"", ""streetNu..."
3,6669b15e44a41007063ba9dd,32.392000,-110.804000,2020-06-22,NASA6,IBAND,S-NPP,UAZ,32.754000,11,...,0.000530,0.35,12,0.35,"{'type': 'Point', 'coordinates': [-110.804, 32...",US,Mount Lemmon,85619,"10000 East Ski Run Road, Mount Lemmon, AZ 85619","{""address"": {""buildingNumber"": ""10000"", ""stree..."
4,6669b15e44a41007063ba9de,33.472000,-91.333200,2020-06-13,NASA7,IBAND,NOAA-20,UAR,31.359000,37,...,0.000499,0.35,8,0.35,"{'type': 'Point', 'coordinates': [-91.3332, 33...",US,Carlton,71653,"300 Cedar Road, Lake Village, AR 71653","{""address"": {""buildingNumber"": ""300"", ""streetN..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449134,6670e69da6dc531d5ed14328,56.340000,-119.229897,2023-05-13,NASA,MODIS,Aqua,AB,25.278999,25,...,0.032534,2.44,9,2.19,"{'type': 'Point', 'coordinates': [-119.229897,...",CA,Worsley,T0H,"Range Road 84, Worsley AB T0H 3W0","{""address"": {""routeNumbers"": [], ""street"": ""Ra..."
449135,6670e69ea6dc531d5ed14329,31.086281,-98.919960,2023-03-26,NASA7,VIIRS-I,NOAA-20,UTX,21.538000,18,...,0.008307,0.35,9,0.35,"{'type': 'Point', 'coordinates': [-98.91996, 3...",US,San Saba,76877,"Deep Creek Road, San Saba, TX 76877","{""address"": {""routeNumbers"": [], ""street"": ""De..."
449136,6670e69ea6dc531d5ed1432a,56.339291,-119.209106,2023-05-13,NASA,MODIS,Aqua,AB,25.422001,25,...,0.217550,2.52,9,2.20,"{'type': 'Point', 'coordinates': [-119.209106,...",CA,Worsley,T0H,"Range Road 84, Worsley AB T0H 3W0","{""address"": {""routeNumbers"": [], ""street"": ""Ra..."
449137,6670e69ea6dc531d5ed1432b,56.049808,-116.394592,2023-05-13,NASA2,VIIRS-I,S-NPP,AB,25.267000,24,...,0.162315,2.38,9,2.00,"{'type': 'Point', 'coordinates': [-116.394592,...",CA,Cadotte Lake,T0H,Cadotte Lake AB T0H 0N0,"{""address"": {""routeNumbers"": [], ""countryCode""..."


In [18]:
null_country_rows = merged_df[merged_df['country'].isnull()]

In [19]:
null_country_rows.shape

(183, 40)

In [20]:
pd.set_option('display.max_colwidth', None)

In [21]:
null_country_rows.head(5)

,_id,lat,lon,rep_date,source,sensor,satellite,agency,temp,rh,...,cfl,tfc0,ecozone,sfc0,geometry,country,municipality,postalCode,freeformAddress,addressComponents
144,6669b17744a41007063baa6a,28.5780,-87.9260,2020-06-14,USFS,MODIS,Aqua,-,30.430,51,...,-1.0,0.00,NaN,0.00,"{'type': 'Point', 'coordinates': [-87.926, 28.578]}",None,None,None,None,"{""address"": {""routeNumbers"": [], ""boundingBox"": {""northEast"": ""89.999900,180.000000"", ""southWest"": ""-85.344420,-180.000000"", ""entity"": ""position""}}, ""position"": ""28.577999,-87.926003"", ""dataSources"": {""geometry"": {""id"": ""00005831-3200-1200-0000-0000773ade4e""}}, ""id"": ""00005831-3200-1200-0000-0000773ade4e""}"
224,6669b18544a41007063baaba,28.9910,-113.1190,2020-07-03,USFS,MODIS,Terra,-,30.789,30,...,-1.0,0.10,NaN,0.10,"{'type': 'Point', 'coordinates': [-113.119, 28.991]}",None,None,None,None,{}
965,6669b20644a41007063bad9f,28.9940,-113.1250,2020-04-19,USFS,MODIS,Terra,-,27.403,32,...,-1.0,0.35,NaN,0.35,"{'type': 'Point', 'coordinates': [-113.125, 28.994]}",None,None,None,None,{}
3931,6669b40a44a41007063bb937,28.5782,-87.9340,2020-07-09,NASA6,IBAND,S-NPP,-,32.351,61,...,-1.0,0.00,NaN,0.00,"{'type': 'Point', 'coordinates': [-87.934, 28.5782]}",None,None,None,None,"{""address"": {""routeNumbers"": [], ""boundingBox"": {""northEast"": ""89.999900,180.000000"", ""southWest"": ""-85.344420,-180.000000"", ""entity"": ""position""}}, ""position"": ""28.578199,-87.933998"", ""dataSources"": {""geometry"": {""id"": ""00005831-3200-1200-0000-0000773ade4e""}}, ""id"": ""00005831-3200-1200-0000-0000773ade4e""}"
7247,6669b64c44a41007063bc62f,28.7543,-88.2708,2020-07-21,NOAA,VIIRS,S-NPP,-,30.272,64,...,-1.0,0.00,NaN,0.00,"{'type': 'Point', 'coordinates': [-88.2708, 28.7543]}",None,None,None,None,"{""address"": {""routeNumbers"": [], ""boundingBox"": {""northEast"": ""89.999900,180.000000"", ""southWest"": ""-85.344420,-180.000000"", ""entity"": ""position""}}, ""position"": ""28.754299,-88.270798"", ""dataSources"": {""geometry"": {""id"": ""00005831-3200-1200-0000-0000773ade4e""}}, ""id"": ""00005831-3200-1200-0000-0000773ade4e""}"


In [25]:
#lon, lat, cfb ,country , addresscomponent
selected_columns = null_country_rows[['lon', 'lat', 'cfb', 'country', 'addressComponents']]
selected_columns

,lon,lat,cfb,country,addressComponents
144,-87.926000,28.578000,0.0,None,"{""address"": {""routeNumbers"": [], ""boundingBox"": {""northEast"": ""89.999900,180.000000"", ""southWest"": ""-85.344420,-180.000000"", ""entity"": ""position""}}, ""position"": ""28.577999,-87.926003"", ""dataSources"": {""geometry"": {""id"": ""00005831-3200-1200-0000-0000773ade4e""}}, ""id"": ""00005831-3200-1200-0000-0000773ade4e""}"
224,-113.119000,28.991000,0.0,None,{}
965,-113.125000,28.994000,0.0,None,{}
3931,-87.934000,28.578200,0.0,None,"{""address"": {""routeNumbers"": [], ""boundingBox"": {""northEast"": ""89.999900,180.000000"", ""southWest"": ""-85.344420,-180.000000"", ""entity"": ""position""}}, ""position"": ""28.578199,-87.933998"", ""dataSources"": {""geometry"": {""id"": ""00005831-3200-1200-0000-0000773ade4e""}}, ""id"": ""00005831-3200-1200-0000-0000773ade4e""}"
7247,-88.270800,28.754300,0.0,None,"{""address"": {""routeNumbers"": [], ""boundingBox"": {""northEast"": ""89.999900,180.000000"", ""southWest"": ""-85.344420,-180.000000"", ""entity"": ""position""}}, ""position"": ""28.754299,-88.270798"", ""dataSources"": {""geometry"": {""id"": ""00005831-3200-1200-0000-0000773ade4e""}}, ""id"": ""00005831-3200-1200-0000-0000773ade4e""}"
...,...,...,...,...,...
430343,-91.405731,28.758060,0.0,None,"{""address"": {""routeNumbers"": [], ""boundingBox"": {""northEast"": ""89.999900,180.000000"", ""southWest"": ""-85.344420,-180.000000"", ""entity"": ""position""}}, ""position"": ""28.758060,-91.405731"", ""dataSources"": {""geometry"": {""id"": ""00005831-3200-1200-0000-0000773ade4e""}}, ""id"": ""00005831-3200-1200-0000-0000773ade4e""}"
430371,-91.405731,28.758011,0.0,None,"{""address"": {""routeNumbers"": [], ""boundingBox"": {""northEast"": ""89.999900,180.000000"", ""southWest"": ""-85.344420,-180.000000"", ""entity"": ""position""}}, ""position"": ""28.758011,-91.405731"", ""dataSources"": {""geometry"": {""id"": ""00005831-3200-1200-0000-0000773ade4e""}}, ""id"": ""00005831-3200-1200-0000-0000773ade4e""}"
430385,-91.404587,28.762220,0.0,None,"{""address"": {""routeNumbers"": [], ""boundingBox"": {""northEast"": ""89.999900,180.000000"", ""southWest"": ""-85.344420,-180.000000"", ""entity"": ""position""}}, ""position"": ""28.762220,-91.404587"", ""dataSources"": {""geometry"": {""id"": ""00005831-3200-1200-0000-0000773ade4e""}}, ""id"": ""00005831-3200-1200-0000-0000773ade4e""}"
432158,-90.363472,27.175211,0.0,None,"{""address"": {""routeNumbers"": [], ""boundingBox"": {""northEast"": ""89.999900,180.000000"", ""southWest"": ""-85.344420,-180.000000"", ""entity"": ""position""}}, ""position"": ""27.175211,-90.363472"", ""dataSources"": {""geometry"": {""id"": ""00005831-3200-1200-0000-0000773ade4e""}}, ""id"": ""00005831-3200-1200-0000-0000773ade4e""}"


In [23]:
only_address_component

NameError: name 'only_address_component' is not defined

In [ ]:
#Connect to MongoDB (Ensure MongoDB is running and replace URI as necessary)
client = MongoClient('MONGO_URI')

In [ ]:
# Get MongoDB URI, database, and collection names from environment variables
mongo_uri = os.getenv('MONGO_URI')
database_name = os.getenv('DB_NAME')
collection_name = os.getenv('COLLECTION_NAME_MERGE') #collection name for merged data only before filtering

print(mongo_uri,database_name,collection_name)

In [ ]:
# Connect to MongoDB using the URI from .env
client = MongoClient(mongo_uri)

# Access the specified database
db = client[database_name]

In [ ]:
print(db)

In [ ]:
# Check if the collection with the name merge exists and this clears all the previous reocords so there is no overlap of datasets
if collection_name in db.list_collection_names():
    collection = db[collection_name]
    if collection.count_documents({}) > 0:
        collection.delete_many({})
        print(f"Cleared existing documents from collection '{collection_name}'.")
else:
    collection = db[collection_name]

In [ ]:
#Code to insert data into mongodb
data_dict = merged_df.to_dict('records')
collection.insert_many(data_dict)

print("Data successfully inserted into MongoDB.")

In [ ]:
'''Filtering steps , here we filter only the data for canada as the country, so converting all the values to lowercase and comparing'''
# Convert the 'country' column to lowercase and filter for 'ca' or 'canada'
merged_df = merged_df[merged_df['country'].str.lower().isin(['ca', 'canada'])]



In [ ]:
print("shape of df with just canada as country",merged_df.shape)


In [ ]:
print("count of the files is :",merged_df.count())

In [ ]:
# Connect to MongoDB (Ensure MongoDB is running and replace URI as necessary)
client = MongoClient('MONGO_URI')

In [ ]:
#pip install python-dotenv

In [ ]:
#load_dotenv()

# Get MongoDB URI, database, and collection names from environment variables
mongo_uri = os.getenv('MONGO_URI')
database_name = os.getenv('DB_NAME')
collection_name = os.getenv('COLLECTION_NAME_MERGE_CLEAN')

print(mongo_uri,database_name,collection_name)


In [ ]:
# Connect to MongoDB using the URI from .env
client = MongoClient(mongo_uri)

# Access the specified database
db = client[database_name]

In [ ]:
print(db)

In [ ]:
# Check if the collection exists and clear it if necessary
if collection_name in db.list_collection_names():
    collection = db[collection_name]
    if collection.count_documents({}) > 0:
        collection.delete_many({})
        print(f"Cleared existing documents from collection '{collection_name}'.")
else:
    collection = db[collection_name]

In [ ]:
# Assuming merged_df is your DataFrame ready to be inserted
# Example DataFrame - replace this with your actual DataFrame
# merged_df = pd.DataFrame(data)  # Ensure you have your DataFrame ready

#dbu.insert_df_only_to_mongodb(merged_df, os.getenv('COLLECTION_NAME_FEATUREENGINEERED'))

data_dict = merged_df.to_dict('records')
collection.insert_many(data_dict)

print("Data successfully inserted into MongoDB.")